In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, roc_curve, auc,confusion_matrix,ConfusionMatrixDisplay,f1_score,recall_score,precision_score
import pickle

In [ ]:
data = pd.read_csv("ppi.csv",index_col=0)
data = data[data.sequence != 'X']

In [ ]:
#One Hot Encoding for sequence, spliting test and train
edata=data
cate_features=[4]
enc = OneHotEncoder(handle_unknown='ignore')
column_transformer = ColumnTransformer(
[('encoder', enc, cate_features)],
remainder='passthrough',
verbose_feature_names_out=False
)
edata = column_transformer.fit_transform(edata)
encoded_df = pd.DataFrame(edata, columns=column_transformer.get_feature_names_out(list(data.columns)))
encoded_df=encoded_df.drop('normalized_hydropathy_index',axis=1)
encoded_df.describe()

y=encoded_df.uniprot_id
y=y.drop_duplicates()
y1=y.sample(frac=1)
y1.iloc[:45]
test_prots1=list(y1.iloc[:45])
train_prots1=list(y1.iloc[45:])
test1 = encoded_df[encoded_df['uniprot_id'].isin(test_prots1)]
train1 = encoded_df[encoded_df['uniprot_id'].isin(train_prots1)]
train1.to_csv('train1.csv', index=False)
test1.to_csv('test1.csv',index=False)

In [ ]:
#cluster sequence feature
data = pd.read_csv("ppi.csv",index_col=0)
data = data[data.sequence != 'X']
cluster_data=data.drop('normalized_hydropathy_index',axis=1)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"].isin(['A','G','V']),1)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"].isin(['I','L','F','P']),2)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"].isin(['Y','M','T','S']),3)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"].isin(['H','N','Q','W']),4)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"].isin(['R','K']),5)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"].isin(['D','E']),6)
cluster_data["sequence"] = cluster_data["sequence"].mask(cluster_data["sequence"]=='C',7)

test_prots2=list(test1.uniprot_id.drop_duplicates())
train_prots2=list(train1.uniprot_id.drop_duplicates())
test2 = cluster_data[cluster_data['uniprot_id'].isin(test_prots2)]
train2 = cluster_data[cluster_data['uniprot_id'].isin(train_prots2)]
train2.to_csv('train2.csv', index=False)
test2.to_csv('test2.csv',index=False)

In [ ]:
#remove sequence feature
data = pd.read_csv("ppi.csv",index_col=0)
data = data[data.sequence != 'X']
removed_data=data
removed_data=removed_data.drop(['sequence','normalized_hydropathy_index'],axis=1)
test4 = removed_data[removed_data['uniprot_id'].isin(test_prots2)]
train4 = removed_data[removed_data['uniprot_id'].isin(train_prots2)]
train4.to_csv('train4.csv', index=False)
test4.to_csv('test4.csv',index=False)

In [ ]:
#generate 5-folds for cv
cv_prots1 = train_prots2[0:37]
cv_prots2 = train_prots2[37:74]
cv_prots3 = train_prots2[74:111]
cv_prots4 = train_prots2[111:147]
cv_prots5 = train_prots2[147:]
fold1 = train1[train1['uniprot_id'].isin(cv_prots1)]
fold2 = train1[train1['uniprot_id'].isin(cv_prots2)]
fold3 = train1[train1['uniprot_id'].isin(cv_prots3)]
fold4 = train1[train1['uniprot_id'].isin(cv_prots4)]
fold5 = train1[train1['uniprot_id'].isin(cv_prots5)]
cv_train1 = pd.concat([fold1, fold2,fold3,fold4], axis=0, ignore_index=True)
cv_test1 = fold5
cv_train2 = pd.concat([fold1, fold2,fold3,fold5], axis=0, ignore_index=True)
cv_test2 = fold4
cv_train3 = pd.concat([fold1, fold2,fold4,fold5], axis=0, ignore_index=True)
cv_test3 = fold3
cv_train4 = pd.concat([fold1, fold3,fold4,fold5], axis=0, ignore_index=True)
cv_test4 = fold2
cv_train5 = pd.concat([fold2, fold3,fold4,fold5], axis=0, ignore_index=True)
cv_test5 = fold1
cv_train1.to_csv('cv_train1.csv', index=False)
cv_test1.to_csv('cv_test1.csv',index=False)
cv_train2.to_csv('cv_train2.csv', index=False)
cv_test2.to_csv('cv_test2.csv',index=False)
cv_train3.to_csv('cv_train3.csv', index=False)
cv_test3.to_csv('cv_test3.csv',index=False)
cv_train4.to_csv('cv_train4.csv', index=False)
cv_test4.to_csv('cv_test4.csv',index=False)
cv_train5.to_csv('cv_train5.csv', index=False)
cv_test5.to_csv('cv_test5.csv',index=False)

fold1 = train2[train2['uniprot_id'].isin(cv_prots1)]
fold2 = train2[train2['uniprot_id'].isin(cv_prots2)]
fold3 = train2[train2['uniprot_id'].isin(cv_prots3)]
fold4 = train2[train2['uniprot_id'].isin(cv_prots4)]
fold5 = train2[train2['uniprot_id'].isin(cv_prots5)]
cv2_train1 = pd.concat([fold1, fold2,fold3,fold4], axis=0, ignore_index=True)
cv2_test1 = fold5
cv2_train2 = pd.concat([fold1, fold2,fold3,fold5], axis=0, ignore_index=True)
cv2_test2 = fold4
cv2_train3 = pd.concat([fold1, fold2,fold4,fold5], axis=0, ignore_index=True)
cv2_test3 = fold3
cv2_train4 = pd.concat([fold1, fold3,fold4,fold5], axis=0, ignore_index=True)
cv2_test4 = fold2
cv2_train5 = pd.concat([fold2, fold3,fold4,fold5], axis=0, ignore_index=True)
cv2_test5 = fold1
cv2_train1.to_csv('cv2_train1.csv', index=False)
cv2_test1.to_csv('cv2_test1.csv',index=False)
cv2_train2.to_csv('cv2_train2.csv', index=False)
cv2_test2.to_csv('cv2_test2.csv',index=False)
cv2_train3.to_csv('cv2_train3.csv', index=False)
cv2_test3.to_csv('cv2_test3.csv',index=False)
cv2_train4.to_csv('cv2_train4.csv', index=False)
cv2_test4.to_csv('cv2_test4.csv',index=False)
cv2_train5.to_csv('cv2_train5.csv', index=False)
cv2_test5.to_csv('cv2_test5.csv',index=False)


fold1 = train4[train4['uniprot_id'].isin(cv_prots1)]
fold2 = train4[train4['uniprot_id'].isin(cv_prots2)]
fold3 = train4[train4['uniprot_id'].isin(cv_prots3)]
fold4 = train4[train4['uniprot_id'].isin(cv_prots4)]
fold5 = train4[train4['uniprot_id'].isin(cv_prots5)]
cv4_train1 = pd.concat([fold1, fold2,fold3,fold4], axis=0, ignore_index=True)
cv4_test1 = fold5
cv4_train2 = pd.concat([fold1, fold2,fold3,fold5], axis=0, ignore_index=True)
cv4_test2 = fold4
cv4_train3 = pd.concat([fold1, fold2,fold4,fold5], axis=0, ignore_index=True)
cv4_test3 = fold3
cv4_train4 = pd.concat([fold1, fold3,fold4,fold5], axis=0, ignore_index=True)
cv4_test4 = fold2
cv4_train5 = pd.concat([fold2, fold3,fold4,fold5], axis=0, ignore_index=True)
cv4_test5 = fold1
cv4_train1.to_csv('cv4_train1.csv', index=False)
cv4_test1.to_csv('cv4_test1.csv',index=False)
cv4_train2.to_csv('cv4_train2.csv', index=False)
cv4_test2.to_csv('cv4_test2.csv',index=False)
cv4_train3.to_csv('cv4_train3.csv', index=False)
cv4_test3.to_csv('cv4_test3.csv',index=False)
cv4_train4.to_csv('cv4_train4.csv', index=False)
cv4_test4.to_csv('cv4_test4.csv',index=False)
cv4_train5.to_csv('cv4_train5.csv', index=False)
cv4_test5.to_csv('cv4_test5.csv',index=False)

In [ ]:
#RUS for cv
cv_train1 = pd.read_csv("cv_train1.csv")
cv_train2 = pd.read_csv("cv_train2.csv")
cv_train3 = pd.read_csv("cv_train3.csv")
cv_train4 = pd.read_csv("cv_train4.csv")
cv_train5 = pd.read_csv("cv_train5.csv")

x = cv_train1.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train1.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv1.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

x = cv_train2.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train2.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv2.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

x = cv_train3.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train3.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv3.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

x = cv_train4.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train4.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv4.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

x = cv_train5.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train5.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv5.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

In [ ]:
cv_train1 = pd.read_csv("cv2_train1.csv")
cv_train2 = pd.read_csv("cv2_train2.csv")
cv_train3 = pd.read_csv("cv2_train3.csv")
cv_train4 = pd.read_csv("cv2_train4.csv")
cv_train5 = pd.read_csv("cv2_train5.csv")
x = cv_train1.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train1.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv21.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

x = cv_train2.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train2.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv22.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

x = cv_train3.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train3.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv23.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

x = cv_train4.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train4.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv24.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

x = cv_train5.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train5.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv25.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

In [ ]:
cv_train1 = pd.read_csv("cv4_train1.csv")
cv_train2 = pd.read_csv("cv4_train2.csv")
cv_train3 = pd.read_csv("cv4_train3.csv")
cv_train4 = pd.read_csv("cv4_train4.csv")
cv_train5 = pd.read_csv("cv4_train5.csv")
x = cv_train1.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train1.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv41.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

x = cv_train2.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train2.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv42.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

x = cv_train3.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train3.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv43.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

x = cv_train4.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train4.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv44.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

x = cv_train5.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train5.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(rus_y))
import pickle
with open("RUScv45.pickle", "wb") as file:
    pickle.dump((rus_x, rus_y), file)

In [ ]:
#Smote for cv
x = cv_train1.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train1.p_interface
smote = SMOTE(random_state=42)
smote1_x,smote1_y = smote.fit_resample(x,y)
print('Resampled dataset shape %s' % Counter(smote1_y))

x = cv_train2.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train2.p_interface
smote = SMOTE(random_state=42)
smote2_x,smote2_y = smote.fit_resample(x,y)
print('Resampled dataset shape %s' % Counter(smote2_y))

x = cv_train3.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train3.p_interface
smote = SMOTE(random_state=42)
smote3_x,smote3_y = smote.fit_resample(x,y)
print('Resampled dataset shape %s' % Counter(smote3_y))

x = cv_train4.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train4.p_interface
smote = SMOTE(random_state=42)
smote4_x,smote4_y = smote.fit_resample(x,y)
print('Resampled dataset shape %s' % Counter(smote4_y))

x = cv_train5.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = cv_train5.p_interface
smote = SMOTE(random_state=42)
smote5_x,smote5_y = smote.fit_resample(x,y)
print('Resampled dataset shape %s' % Counter(smote5_y))

with open("SMOTE.pickle", "wb") as file:
    pickle.dump((smote1_x, smote1_y,smote2_x,smote2_y,smote3_x,smote3_y,smote4_x,smote4_y,smote5_x,smote5_y), file)

In [ ]:
#RUS for whole train set(used for final model)
train1 = pd.read_csv("train1.csv")
x = train1.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y = train1.p_interface
rus = RandomUnderSampler(random_state=36)
rus_x,rus_y = rus.fit_resample(x, y)
rus_x.to_csv('rus_x.csv', index=False)
rus_y.to_csv('rus_y.csv',index=False)

In [ ]:
#load data for hyperparameter tuning
cv_train1 = pd.read_csv("cv_train1.csv")
x_train1 = cv_train1.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y_train1 = cv_train1.p_interface

cv_train2 = pd.read_csv("cv_train2.csv")
x_train2 = cv_train2.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y_train2 = cv_train2.p_interface

cv_train3 = pd.read_csv("cv_train3.csv")
x_train3 = cv_train3.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y_train3 = cv_train3.p_interface

cv_train4 = pd.read_csv("cv_train4.csv")
x_train4 = cv_train4.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y_train4 = cv_train4.p_interface

cv_train5 = pd.read_csv("cv_train5.csv")
x_train5 = cv_train5.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y_train5 = cv_train5.p_interface

cv_test1 = pd.read_csv("cv_test1.csv")
cv_test2 = pd.read_csv("cv_test2.csv")
cv_test3 = pd.read_csv("cv_test3.csv")
cv_test4 = pd.read_csv("cv_test4.csv")
cv_test5 = pd.read_csv("cv_test5.csv")

x_test1 = cv_test1.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y_test1 = cv_test1.p_interface
x_test2 = cv_test2.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y_test2 = cv_test2.p_interface
x_test3 = cv_test3.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y_test3 = cv_test3.p_interface
x_test4 = cv_test4.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y_test4 = cv_test4.p_interface
x_test5 = cv_test5.drop(['aa_ProtPosition', 'domain','p_interface','uniprot_id','Rlength'], axis=1)
y_test5 = cv_test5.p_interface

In [ ]:
#grid search(same setting for all except final model)
for i in [100,120,140,160]:
    for j in [10,15,20]:
        rf = RandomForestClassifier(random_state=0,n_estimators=i,max_depth=j,n_jobs=-1)
        rf.fit(x_train1, y_train1)
        rf_predictions = rf.predict(x_test1)
        f1_1 = f1_score(y_test1,rf_predictions)
        acc_1 = accuracy_score(y_test1,rf_predictions)
        y_rf_prob = rf.predict_proba(x_test1)[:, 1]
        roc_rf = roc_curve(y_test1, y_rf_prob)
        auc_1 = auc(roc_rf[0], roc_rf[1])
        
        rf = RandomForestClassifier(random_state=0,n_estimators=i,max_depth=j,n_jobs=-1)
        rf.fit(x_train2, y_train2)
        rf_predictions = rf.predict(x_test2)
        f1_2 = f1_score(y_test2,rf_predictions)
        acc_2 = accuracy_score(y_test2,rf_predictions)
        y_rf_prob = rf.predict_proba(x_test2)[:, 1]
        roc_rf = roc_curve(y_test2, y_rf_prob)
        auc_2 = auc(roc_rf[0], roc_rf[1])
        
        rf = RandomForestClassifier(random_state=0,n_estimators=i,max_depth=j,n_jobs=-1)
        rf.fit(x_train3, y_train3)
        rf_predictions = rf.predict(x_test3)
        f1_3 = f1_score(y_test3,rf_predictions)
        acc_3 = accuracy_score(y_test3,rf_predictions)
        y_rf_prob = rf.predict_proba(x_test3)[:, 1]
        roc_rf = roc_curve(y_test3, y_rf_prob)
        auc_3 = auc(roc_rf[0], roc_rf[1])
        
        rf = RandomForestClassifier(random_state=0,n_estimators=i,max_depth=j,n_jobs=-1)
        rf.fit(x_train4, y_train4)
        rf_predictions = rf.predict(x_test4)
        f1_4 = f1_score(y_test4,rf_predictions)
        acc_4 = accuracy_score(y_test4,rf_predictions)
        y_rf_prob = rf.predict_proba(x_test4)[:, 1]
        roc_rf = roc_curve(y_test4, y_rf_prob)
        auc_4 = auc(roc_rf[0], roc_rf[1])
        
        rf = RandomForestClassifier(random_state=0,n_estimators=i,max_depth=j,n_jobs=-1)
        rf.fit(x_train5, y_train5)
        rf_predictions = rf.predict(x_test5)
        f1_5 = f1_score(y_test5,rf_predictions)
        acc_5 = accuracy_score(y_test5,rf_predictions)
        y_rf_prob = rf.predict_proba(x_test5)[:, 1]
        roc_rf = roc_curve(y_test5, y_rf_prob)
        auc_5 = auc(roc_rf[0], roc_rf[1])
        
        print(i,j,(f1_1+f1_2+f1_3+f1_4+f1_5)/5,(acc_1+acc_2+acc_3+acc_4+acc_5)/5,(auc_1+auc_2+auc_3+auc_4+auc_5)/5)

In [ ]:
#evaluation of feature engineering
#original sequence
with open("RUScv1.pickle", "rb") as file:
    rus1_x, rus1_y = pickle.load(file)

with open("RUScv2.pickle", "rb") as file:
    rus2_x, rus2_y = pickle.load(file)

with open("RUScv3.pickle", "rb") as file:
    rus3_x, rus3_y = pickle.load(file)

with open("RUScv4.pickle", "rb") as file:
    rus4_x, rus4_y = pickle.load(file)

with open("RUScv5.pickle", "rb") as file:
    rus5_x, rus5_y = pickle.load(file)
for i in [100,120,140,160]:
    for j in [10,15,20]:
        rf = RandomForestClassifier(random_state=0,n_estimators=i,max_depth=j,n_jobs=-1)
        rf.fit(rus1_x, rus1_y)
        rf_predictions = rf.predict(x_test1)
        f1_1 = f1_score(y_test1,rf_predictions)
        rf.fit(rus2_x, rus2_y)
        rf_predictions = rf.predict(x_test2)
        f1_2 = f1_score(y_test2,rf_predictions)
        rf.fit(rus3_x, rus3_y)
        rf_predictions = rf.predict(x_test3)
        f1_3 = f1_score(y_test3,rf_predictions)
        rf.fit(rus4_x, rus4_y)
        rf_predictions = rf.predict(x_test4)
        f1_4 = f1_score(y_test4,rf_predictions)
        rf.fit(rus5_x, rus5_y)
        rf_predictions = rf.predict(x_test5)
        f1_5 = f1_score(y_test5,rf_predictions)
        
        print(i,j,(f1_1+f1_2+f1_3+f1_4+f1_5)/5)
#cluster sequence
with open("RUScv21.pickle", "rb") as file:
    rus1_x, rus1_y = pickle.load(file)

with open("RUScv22.pickle", "rb") as file:
    rus2_x, rus2_y = pickle.load(file)

with open("RUScv23.pickle", "rb") as file:
    rus3_x, rus3_y = pickle.load(file)

with open("RUScv24.pickle", "rb") as file:
    rus4_x, rus4_y = pickle.load(file)

with open("RUScv25.pickle", "rb") as file:
    rus5_x, rus5_y = pickle.load(file)
for i in [100,120,140,160]:
    for j in [10,15,20]:
        rf = RandomForestClassifier(random_state=0,n_estimators=i,max_depth=j,n_jobs=-1)
        rf.fit(rus1_x, rus1_y)
        rf_predictions = rf.predict(x_test1)
        f1_1 = f1_score(y_test1,rf_predictions)
        rf.fit(rus2_x, rus2_y)
        rf_predictions = rf.predict(x_test2)
        f1_2 = f1_score(y_test2,rf_predictions)
        rf.fit(rus3_x, rus3_y)
        rf_predictions = rf.predict(x_test3)
        f1_3 = f1_score(y_test3,rf_predictions)
        rf.fit(rus4_x, rus4_y)
        rf_predictions = rf.predict(x_test4)
        f1_4 = f1_score(y_test4,rf_predictions)
        rf.fit(rus5_x, rus5_y)
        rf_predictions = rf.predict(x_test5)
        f1_5 = f1_score(y_test5,rf_predictions)
        
        print(i,j,(f1_1+f1_2+f1_3+f1_4+f1_5)/5)
#remove sequence
with open("RUScv41.pickle", "rb") as file:
    rus1_x, rus1_y = pickle.load(file)

with open("RUScv42.pickle", "rb") as file:
    rus2_x, rus2_y = pickle.load(file)

with open("RUScv43.pickle", "rb") as file:
    rus3_x, rus3_y = pickle.load(file)

with open("RUScv44.pickle", "rb") as file:
    rus4_x, rus4_y = pickle.load(file)

with open("RUScv45.pickle", "rb") as file:
    rus5_x, rus5_y = pickle.load(file)
for i in [100,120,140,160]:
    for j in [10,15,20]:
        rf = RandomForestClassifier(random_state=0,n_estimators=i,max_depth=j,n_jobs=-1)
        rf.fit(rus1_x, rus1_y)
        rf_predictions = rf.predict(x_test1)
        f1_1 = f1_score(y_test1,rf_predictions)
        rf.fit(rus2_x, rus2_y)
        rf_predictions = rf.predict(x_test2)
        f1_2 = f1_score(y_test2,rf_predictions)
        rf.fit(rus3_x, rus3_y)
        rf_predictions = rf.predict(x_test3)
        f1_3 = f1_score(y_test3,rf_predictions)
        rf.fit(rus4_x, rus4_y)
        rf_predictions = rf.predict(x_test4)
        f1_4 = f1_score(y_test4,rf_predictions)
        rf.fit(rus5_x, rus5_y)
        rf_predictions = rf.predict(x_test5)
        f1_5 = f1_score(y_test5,rf_predictions)
        
        print(i,j,(f1_1+f1_2+f1_3+f1_4+f1_5)/5)

In [ ]:
#recursive feature elimination
x = pd.read_csv("rus_x.csv")
y = pd.read_csv("rus_y.csv")
x=x.drop('normalized_hydropathy_index',axis=1)
y=y.to_numpy().ravel()
features=[]
scores=[]
for i in range(150):
    rf=RandomForestClassifier(random_state=0,n_estimators=140,max_depth=10,n_jobs=-1)
    rf.fit(x,y)
    feature_importance = zip(rf.feature_names_in_,rf.feature_importances_)
    features.append(rf.feature_names_in_)
    rf.fit(rus1_x, rus1_y)
    rf_predictions = rf.predict(x_test1)
    f1_1 = f1_score(y_test1,rf_predictions)
    rf.fit(rus2_x, rus2_y)
    rf_predictions = rf.predict(x_test2)
    f1_2 = f1_score(y_test2,rf_predictions)
    rf.fit(rus3_x, rus3_y)
    rf_predictions = rf.predict(x_test3)
    f1_3 = f1_score(y_test3,rf_predictions)
    rf.fit(rus4_x, rus4_y)
    rf_predictions = rf.predict(x_test4)
    f1_4 = f1_score(y_test4,rf_predictions)
    rf.fit(rus5_x, rus5_y)
    rf_predictions = rf.predict(x_test5)
    f1_5 = f1_score(y_test5,rf_predictions)
    mean_f=(f1_1+f1_2+f1_3+f1_4+f1_5)/5
    scores.append(mean_f)
    lst =list(feature_importance)
    lst = dict(lst)
    min_key = min(lst, key=lst.get)
    if len(x.columns)>1:
        x=x.drop(min_key,axis=1)
        rus1_x=rus1_x.drop(min_key,axis=1)
        rus2_x=rus2_x.drop(min_key,axis=1)
        rus3_x=rus3_x.drop(min_key,axis=1)
        rus4_x=rus4_x.drop(min_key,axis=1)
        rus5_x=rus5_x.drop(min_key,axis=1)
        x_test1=x_test1.drop(min_key,axis=1)
        x_test2=x_test2.drop(min_key,axis=1)
        x_test3=x_test3.drop(min_key,axis=1)
        x_test4=x_test4.drop(min_key,axis=1)
        x_test5=x_test5.drop(min_key,axis=1)

number_of_features=[i for i in range(150,0,-1)]
plt.plot(number_of_features,scores)
plt.xlabel("number of features")
plt.ylabel("mean f1 score")
plt.show()

In [ ]:
#final model training and testing
x = pd.read_csv("rus_x.csv")
y = pd.read_csv("rus_y.csv")
y=y.to_numpy().ravel()
rf = RandomForestClassifier(random_state=0)
params = {'n_estimators': 140, 'max_depth': 10, 'criterion': 'gini'}
rf.set_params(**params)

rf.fit(x,y)
rf_predictions = rf.predict(x_test1)
print("Accuracy of Random Forest:", accuracy_score(y_test1, rf_predictions))
y_rf_prob = rf.predict_proba(x_test1)[:, 1]
roc_rf = roc_curve(y_test1, y_rf_prob)
print("AUC of Random Forest:", auc(roc_rf[0], roc_rf[1]))
f1 = f1_score(y_test1,rf_predictions)
print("f1 score of Random Forest:",f1)
cm = confusion_matrix(y_test1, rf_predictions, labels=rf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=rf.classes_)
disp.plot()
plt.show()


In [ ]:
#feature importance
feature_importance = zip(rf.feature_names_in_,rf.feature_importances_)
lst =list(feature_importance)
lst = dict(lst)
keys = list(lst.keys())
vals = list(lst.values())

lst['wm_normalized_abs_surf_acc'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_normalized_abs_surf_acc'):
        lst['wm_normalized_abs_surf_acc'] += lst[keys[i]]


lst['wm_normalized_hydropathy_index'] = 0
for i in  range(len(keys)):
    if keys[i].endswith('wm_normalized_hydropathy_index'):
        lst['wm_normalized_hydropathy_index'] += lst[keys[i]]


lst['wm_rel_surf_acc'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_rel_surf_acc'):
        lst['wm_rel_surf_acc'] += lst[keys[i]]


lst['wm_prob_sheet'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_prob_sheet'):
        lst['wm_prob_sheet'] += lst[keys[i]]


lst['wm_prob_helix'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_prob_helix'):
        lst['wm_prob_helix'] += lst[keys[i]]


lst['wm_prob_coil'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_prob_coil'):
        lst['wm_prob_coil'] += lst[keys[i]]


lst['wm_pssm_A'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_A'):
        lst['wm_pssm_A'] += lst[keys[i]]


lst['wm_pssm_R'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_R'):
        lst['wm_pssm_R'] += lst[keys[i]]


lst['wm_pssm_N'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_N'):
        lst['wm_pssm_N'] += lst[keys[i]]


lst['wm_pssm_D'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_D'):
        lst['wm_pssm_D'] += lst[keys[i]]


lst['wm_pssm_C'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_C'):
        lst['wm_pssm_C'] += lst[keys[i]]


lst['wm_pssm_Q'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_Q'):
        lst['wm_pssm_Q'] += lst[keys[i]]


lst['wm_pssm_E'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_E'):
        lst['wm_pssm_E'] += lst[keys[i]]


lst['wm_pssm_G'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_G'):
        lst['wm_pssm_G'] += lst[keys[i]]


lst['wm_pssm_H'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_H'):
        lst['wm_pssm_H'] += lst[keys[i]]


lst['wm_pssm_I'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_I'):
        lst['wm_pssm_I'] += lst[keys[i]]


lst['wm_pssm_L'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_L'):
        lst['wm_pssm_L'] += lst[keys[i]]


lst['wm_pssm_K'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_K'):
        lst['wm_pssm_K'] += lst[keys[i]]


lst['wm_pssm_M'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_M'):
        lst['wm_pssm_M'] += lst[keys[i]]


lst['wm_pssm_F'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_F'):
        lst['wm_pssm_F'] += lst[keys[i]]


lst['wm_pssm_P'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_P'):
        lst['wm_pssm_P'] += lst[keys[i]]


lst['wm_pssm_S'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_S'):
        lst['wm_pssm_S'] += lst[keys[i]]


lst['wm_pssm_T'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_T'):
        lst['wm_pssm_T'] += lst[keys[i]]


lst['wm_pssm_W'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_W'):
        lst['wm_pssm_W'] += lst[keys[i]]


lst['wm_pssm_Y'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_Y'):
        lst['wm_pssm_Y'] += lst[keys[i]]


lst['wm_pssm_V'] = 0
for i in range(len(keys)):
    if keys[i].endswith('wm_pssm_V'):
        lst['wm_pssm_V'] += lst[keys[i]]

lst['sum_sequence'] = 0
for i in range(len(keys)):
    if keys[i].startswith('sequence'):
        lst['sum_sequence'] += lst[keys[i]]

for i in range(len(keys)):
    if keys[i].startswith('3_wm') or keys[i].startswith('5_wm') or keys[i].startswith('7_wm') or keys[i].startswith('9_wm') or keys[i].startswith('sequence'):
        del lst[keys[i]]

lst["sequence"] = lst.pop("sum_sequence")

names = list(lst.keys())
importance = list(lst.values())
sorted_idx = np.argsort(importance)
fig = plt.figure(figsize=(9, 11))
plt.barh(range(len(sorted_idx)), np.array(importance)[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(names)[sorted_idx])
plt.title('Feature Importance of Final Model')